In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
df = pd.read_excel('AdvanceData.xlsx')

In [ ]:
def check_distribution_and_variance(df, target_col, split_col, group1_val, group2_val):
    """
    2群の分布（ヒストグラム・箱ひげ図）と分散（Levene検定）を確認する関数
    """
    
    # データの抽出
    # 数値/文字列の型の違いを吸収するための処理
    try:
        data1 = df[df[split_col] == group1_val][target_col].dropna()
        data2 = df[df[split_col] == group2_val][target_col].dropna()
        
        # もしデータが空なら、型変換して再トライ
        if len(data1) == 0 and len(data2) == 0:
            if isinstance(group1_val, int):
                group1_val, group2_val = str(group1_val), str(group2_val)
            else:
                group1_val, group2_val = int(group1_val), int(group2_val)
            data1 = df[df[split_col] == group1_val][target_col].dropna()
            data2 = df[df[split_col] == group2_val][target_col].dropna()
    except:
        print("データ抽出中にエラーが発生しました。カラム名や値を確認してください。")
        return

    # データ件数チェック
    if len(data1) < 2 or len(data2) < 2:
        print("エラー: データ数が不足しています。")
        return

    # グラフの描画設定
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # 結合データフレーム作成（描画用）
    plot_df = df[df[split_col].isin([group1_val, group2_val])]

    # 1. ヒストグラム + KDE（カーネル密度推定）
    # 分布の重なりや形状の違いを直感的に把握できます
    sns.histplot(data=plot_df, x=target_col, hue=split_col, kde=True, element="step", ax=axes[0], palette="Set2")
    axes[0].set_title(f'Distribution Histogram: {target_col}')
    
    # 2. 箱ひげ図 (Boxplot)
    # 中央値、四分位範囲、外れ値、そして「箱の長さ（＝分散の大きさ）」を比較します
    sns.boxplot(data=plot_df, x=split_col, y=target_col, hue=split_col, legend=False,ax=axes[1], palette="Set2")
    axes[1].set_title(f'Boxplot (Variance Check): {target_col}')
    
    plt.tight_layout()
    plt.show()

    # --- 統計的なチェック ---
    print(f"\n【統計的検定: {target_col} by {split_col}】")
    
    # 1. 基本統計量
    var1 = data1.var(ddof=1)
    var2 = data2.var(ddof=1)
    print(f"群1 ({group1_val}): N={len(data1)}, 分散={var1:.4f}, 標準偏差={data1.std():.4f}")
    print(f"群2 ({group2_val}): N={len(data2)}, 分散={var2:.4f}, 標準偏差={data2.std():.4f}")
    
    # 2. Levene検定（等分散性の検定）
    # 正規分布に従わないデータでも比較的堅牢に分散の違いを検定できます
    stat, p_val = stats.levene(data1, data2)
    
    print("-" * 40)
    print(f"Leveneの検定結果 (等分散性の確認)")
    print(f"F値: {stat:.4f}")
    print(f"p値: {p_val:.4f}")
    
    if p_val < 0.05:
        print(">> 判定: 分散に有意差あり (等分散ではない)")
        print("   → Studentのt検定ではなく、「Welchのt検定」を使用すべきです。")
    else:
        print(">> 判定: 分散に有意差なし (等分散とみなせる)")
        print("   → Studentのt検定も使用可能ですが、安全のためWelchの使用を推奨します。")
    print("-" * 40)

# --- 実行例 ---
if __name__ == "__main__":

    # 実行
    check_distribution_and_variance(df, 'UWES_mean', 'gender', 0, 1)

In [ ]:
def calculate_cohens_d(group1, group2):
    """効果量 Cohen's d を計算する関数"""
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    
    # 統合分散（Pooled Variance）
    pooled_se = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
    
    # 平均の差 / 統合分散
    return (np.mean(group1) - np.mean(group2)) / pooled_se

In [ ]:
def analyze_dataframe_welch(df, target_col, split_col, group1_val, group2_val):
    """
    df: データフレーム
    target_col: 検定したい数値データのカラム名（例: 'speech', 'BADL'）
    split_col: 群分けに使うカラム名（例: 'gender', 'high_stress_flag'）
    group1_val: 群1の値（例: 'Male', 1）
    group2_val: 群2の値（例: 'Female', 0）
    """
    
    # データの抽出
    data1 = df[df[split_col] == group1_val][target_col].dropna()
    data2 = df[df[split_col] == group2_val][target_col].dropna()
    
    # データ件数の確認
    n1, n2 = len(data1), len(data2)
    print(f"【検定対象: {target_col}】")
    print(f"群1({group1_val}): N={n1}, Mean={data1.mean():.2f}, SD={data1.std():.2f}")
    print(f"群2({group2_val}): N={n2}, Mean={data2.mean():.2f}, SD={data2.std():.2f}")

    if n1 < 2 or n2 < 2:
        print("エラー: データ数が不足しているため検定できません。")
        return

    # Welchのt検定 (equal_var=False)
    t_stat, p_val = stats.ttest_ind(data1, data2, equal_var=False)
    
    # 効果量 (Cohen's d)
    d = calculate_cohens_d(data1, data2)
    
    print(f"t値: {t_stat:.3f}")
    print(f"p値: {p_val:.4f}")
    print(f"効果量(d): {d:.3f}")
    
    star = ""
    if p_val < 0.01: star = "**"
    elif p_val < 0.05: star = "*"
    
    if star:
        print(f"判定: 有意差あり {star}")
    else:
        print("判定: 有意差なし")

In [ ]:
analyze_dataframe_welch(df, 'env', 'role', 0, 1)